In [1]:
import os
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
sns.set(style="ticks", color_codes=True)
sns.set(rc={'figure.figsize':(10, 10)})
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
le = LabelEncoder()
sc = MinMaxScaler()
sd = StandardScaler()
rs = RobustScaler()
poly = PolynomialFeatures(2)
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef
from sklearn import tree
# from sklearn.tree import export_graphviz

import xgboost as xgb
import lightgbm as LGBM

from subprocess import call
from IPython.display import Image

from custom_functions import (
    gen_MonthDayHrMinDayOfWeek,
    gen_timeblock
)


#### notes
> 提交的預測值必須為 boolean

`users.csv`
- age <0 或 >65 有高比率不開信 可能是填年齡時亂填 不是真實年紀

`train.csv`
- 有一部分 user_id 有被重複寄到 EDM
- 有一部分 user_id 完全沒開過信/登入/購買
    - `isNeverOpen`
    - `isNeverLogin`
    - `isNeverCheckout`
    
- `day` 對開信率有影響
- `attr_2` 暫時排除(測試EDA結果)

`test.csv`
- 全部 user_id 都是一人一封信


--------

##### 測試純條件判斷
`open_flag`==0:
1. `open_count_last_10_days` == 0 or
<!-- 2. `open_count_last_10_days` > 0 and `attr_2_1` == 1 -->


##### worklog
- 2020.08.01 data preprocessing
- 2020.08.02 RF baseline model (0.48)
- 2020.08.06 DNN model (0.51)
- 2020.08.07 polynomial features (not so good)
- 2020.08.08 renew EDA & features 
    - learn from https://www.kaggle.com/marcellosusanto/shopee-ma-eda


In [3]:
### preprocessing 過程
# train.groupby(['last_checkout_day']).size()
# tmp = train[train['last_checkout_day']!='Never checkout']
# tmp['last_checkout_day'] = tmp['last_checkout_day'].astype(int)
# tmp.groupby(['last_checkout_day']).size().reset_index().sort_values('last_checkout_day', ascending=False)


# train = pd.read_csv('train.csv')
train.groupby(['open_flag']).size()
# train_userAttr[(train_userAttr['open_count_last_10_days']==0)].groupby(['open_flag']).size() #|((train_userAttr['open_count_last_10_days']>0)&(train_userAttr['attr_2_1']==1))
# train_userAttr[(train_userAttr['open_count_last_10_days']==0)&(train_userAttr['attr_2_1']==0)&(train_userAttr['open_flag'] == 1)]
# train[(train['last_open_day'] == 'Never open')&(train['last_login_day'] == 'Never login')&(train['last_checkout_day'] == 'Never checkout')&(train['open_flag'] == 1)]
train[(train['open_count_last_10_days']>0)&(train['open_count_trend_60-30']>0)].groupby('open_flag').size()
train[(train['open_count_last_10_days']>0)&(train['open_count_trend_60-30']<=0)].groupby('open_flag').size()


NameError: name 'train' is not defined

In [2]:
user_attr = pd.read_csv('users.csv')
user_attr['attr_1'] = user_attr['attr_1'].fillna(-1)
user_attr['attr_2'] = user_attr['attr_2'].fillna(-1)
user_attr['age'] = user_attr['age'].fillna(-1)
user_attr['age_type'] = np.where((user_attr['age']!=-1)&(user_attr['age']<30), '<30', '>=30')
user_attr['age_type'] = np.where((user_attr['age']==-1), 'unknown', user_attr['age_type'])
# user_attr['age'] = np.where(user_attr['age']>=30, '>=30', user_attr['age'])
# user_attr['age'] = user_attr['age'].astype('str')
# user_attr['domain'] = le.fit_transform(user_attr['domain'])
# user_attr['domain'] = user_attr['domain'].astype('int64')
user_attr['attr_1'] = user_attr['attr_1'].astype('int64').astype(str)
user_attr['attr_2'] = user_attr['attr_2'].astype('int64').astype(str)
user_attr['attr_3'] = user_attr['attr_3'].astype('int64').astype(str)
# user_attr.drop(['attr_1','attr_1_-1','attr_2','attr_2_-1','attr_3','attr_3_0','domain','domain_other'], axis=1, inplace=True)

list_low_domain = ['@163.com','@gmail.com','@yahoo.com','@ymail.com']
list_med_domain = ['@outlook.com','@qq.com','@rocketmail.com']
list_high_domain = ['@hotmail.com','@icloud.com','@live.com','other']
cond_list = [
    user_attr['domain'].isin(list_low_domain),
    user_attr['domain'].isin(list_med_domain),
    user_attr['domain'].isin(list_high_domain)
]
choic_list = [
    'low_domain', 'med_domain', 'high_domain'
]
user_attr['domain_type'] = np.select(cond_list, choic_list, default='not_cated')
user_attr[user_attr['domain_type']=='not_cated']
dummy_userAttr = pd.get_dummies(user_attr[['age_type','attr_1','attr_3','domain_type']])
user_attr = pd.concat([user_attr, dummy_userAttr], axis=1)
user_attr.drop(['age','age_type','age_type_unknown','attr_1','attr_1_-1','attr_2','attr_3','attr_3_0','domain','domain_type','domain_type_low_domain'], axis=1, inplace=True)
# user_attr.info()
# user_attr.isnull().sum()
user_attr.shape
user_attr.head()

train = pd.read_csv('train.csv')
train['last_open_day'] = np.where(train['last_open_day']=='Never open', max([int(x) for x in train['last_open_day'].values if x != 'Never open']), train['last_open_day'])
train['last_login_day'] = np.where(train['last_login_day']=='Never login', max([int(x) for x in train['last_login_day'].values if x != 'Never login']), train['last_login_day'])
train['last_checkout_day'] = np.where(train['last_checkout_day']=='Never checkout', max([int(x) for x in train['last_checkout_day'].values if x != 'Never checkout']), train['last_checkout_day'])# train['last_open_day'] = np.where(train['last_open_day']=='Never open', 808*10, train['last_open_day'])
train['isNeverOpen'] = np.where(train['last_open_day']=='Never open', 1, 0)
train['isNeverLogin'] = np.where(train['last_login_day']=='Never login', 1, 0)
train['isNeverCheckout'] = np.where(train['last_checkout_day']=='Never checkout', 1, 0)
train['last_open_day'] = train['last_open_day'].astype('int64')
train['last_login_day'] = train['last_login_day'].astype('int64')
train['last_checkout_day'] = train['last_checkout_day'].astype('int64')
train = gen_MonthDayHrMinDayOfWeek(train, 'grass_date')
train.drop(['minute','month','hour'], axis=1, inplace=True)
train['day'] = train['day'].astype(int)
train['country_code'] = train['country_code'].astype(str)
dummy_train = pd.get_dummies(train[['country_code']])
train = pd.concat([train, dummy_train], axis=1)
train.drop(['country_code','country_code_7'], axis=1, inplace=True)
# train.info()
# train.isnull().sum()
train.shape
train.head()
Y = train['open_flag'].values


train_userAttr = pd.merge(train, user_attr, on='user_id', how='left')
train_userAttr.shape
train_userAttr.head()
train_userAttr.groupby(['open_flag']).size()
# train_userAttr.isnull().sum()


test = pd.read_csv('test.csv')
# test = test.sort_values('user_id')
test['last_open_day'] = np.where(test['last_open_day']=='Never open', max([int(x) for x in test['last_open_day'].values if x != 'Never open']), test['last_open_day'])
test['last_login_day'] = np.where(test['last_login_day']=='Never login', max([int(x) for x in test['last_login_day'].values if x != 'Never login']), test['last_login_day'])
test['last_checkout_day'] = np.where(test['last_checkout_day']=='Never checkout', max([int(x) for x in test['last_checkout_day'].values if x != 'Never checkout']), test['last_checkout_day'])
test['isNeverOpen'] = np.where(test['last_open_day']=='Never open', 1, 0)
test['isNeverLogin'] = np.where(test['last_login_day']=='Never login', 1, 0)
test['isNeverCheckout'] = np.where(test['last_checkout_day']=='Never checkout', 1, 0)
test['last_open_day'] = test['last_open_day'].astype('int64')
test['last_login_day'] = test['last_login_day'].astype('int64')
test['last_checkout_day'] = test['last_checkout_day'].astype('int64')
test = gen_MonthDayHrMinDayOfWeek(test, 'grass_date')
test.drop(['minute','month','hour'], axis=1, inplace=True)
test['day'] = test['day'].astype(int)
test['country_code'] = test['country_code'].astype(str)
dummy_test = pd.get_dummies(test[['country_code']])
test = pd.concat([test, dummy_test], axis=1)
test.drop(['country_code','country_code_7'], axis=1, inplace=True)
# test.info()
# test.isnull().sum()
test.shape
test.head()

test_userAttr = pd.merge(test, user_attr, on='user_id', how='left')
test_userAttr.shape
test_userAttr.head()
# test_userAttr.isnull().sum()


submitSample = pd.read_csv('sample_submission_0_1.csv')
submitSample['row_id'] = test['row_id']
submitSample.shape
submitSample.head()



,user_id,attr_1,attr_2,attr_3,age,domain,age_type,domain_type


(127886, 11)

,user_id,age_type_<30,age_type_>=30,attr_1_0,attr_1_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_type_high_domain,domain_type_med_domain
0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,1,0,0,0,0
2,2,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,0
4,4,0,1,0,1,0,1,0,0,0,0


(73539, 27)

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6
0,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0,0,0,0,16,2,0,0,0,1,0,0
1,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1,0,0,0,16,2,0,0,0,1,0,0
2,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2,0,0,0,16,2,0,0,0,0,0,1
3,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3,0,0,0,16,2,1,0,0,0,0,0
4,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4,0,0,0,16,2,0,0,0,0,0,1


...

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6
0,0,35,27,2,13,2,3,4,10,34,134,0,6,18,0,0,0,0,3,2,0,0,0,0,0,1
1,130,35,7,5,383,1,1,1,5,5,5,0,0,0,1,0,0,0,3,2,0,0,0,0,0,1
2,150,25,34,1,3,0,0,0,13,19,38,2,2,2,2,0,0,0,3,2,0,0,0,0,1,0
3,181,36,63,5,5,0,0,0,43,110,173,2,5,5,3,0,0,0,3,2,1,0,0,0,0,0
4,192,23,6,5,54,0,0,0,4,12,39,0,0,2,4,0,0,0,3,2,0,0,0,0,1,0


(55970, 36)

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age_type_<30,age_type_>=30,attr_1_0,attr_1_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_type_high_domain,domain_type_med_domain
0,0,35,27,2,13,2,3,4,10,34,134,0,6,18,0,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,130,35,7,5,383,1,1,1,5,5,5,0,0,0,1,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,150,25,34,1,3,0,0,0,13,19,38,2,2,2,2,0,0,0,3,2,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
3,181,36,63,5,5,0,0,0,43,110,173,2,5,5,3,0,0,0,3,2,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0
4,192,23,6,5,54,0,0,0,4,12,39,0,0,2,4,0,0,0,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


(55970, 2)

,row_id,open_flag
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


#### domain_knowledge features
- `open_count_per10day_last_60_days`
- `open_count_per10day_last_30_days`
- `open_count_trend_60-30`
- `open_count_trend_30-10`

(login, checkout the same)

- `open_per_login`
- `open_per_checkout`


In [3]:
def pca_feature_transform(train_target, feature_name, n_components=10):
    pca = PCA(n_components=n_components)
    target_cols = [f'{feature_name}_count_last_10_days', f'{feature_name}_count_last_30_days', f'{feature_name}_count_last_60_days']
    _PCA = pca.fit_transform(train_target[target_cols].values)
    _PCA = pd.DataFrame(_PCA, columns=[f'{feature_name}_PC_' + str(x) for x in range(0, n_components)])
    train_target = pd.concat([train_target, _PCA], axis=1)
#     train_target = train_target.drop(target_cols, axis=1)
    print(feature_name, pca.explained_variance_ratio_)
    return train_target

def open_count_features(train_userAttr):
    train_userAttr['open_count_per10day_last_60_days'] = train_userAttr['open_count_last_60_days'] / 6
    train_userAttr['open_count_per10day_last_30_days'] = train_userAttr['open_count_last_30_days'] / 3
    train_userAttr['open_count_trend_60-30'] = train_userAttr['open_count_per10day_last_30_days'] - train_userAttr['open_count_per10day_last_60_days']
    train_userAttr['open_count_trend_30-10'] = train_userAttr['open_count_last_10_days'] - train_userAttr['open_count_per10day_last_30_days']
    return train_userAttr

def login_count_features(train_userAttr):
    train_userAttr['open_per_login'] = train_userAttr['open_PC_0'] / train_userAttr['login_PC_0']
    train_userAttr['login_count_per10day_last_60_days'] = train_userAttr['login_count_last_60_days'] / 6
    train_userAttr['login_count_per10day_last_30_days'] = train_userAttr['login_count_last_30_days'] / 3
    train_userAttr['login_count_trend_60-30'] = train_userAttr['login_count_per10day_last_30_days'] - train_userAttr['login_count_per10day_last_60_days']
    train_userAttr['login_count_trend_30-10'] = train_userAttr['login_count_last_10_days'] - train_userAttr['login_count_per10day_last_30_days']
    return train_userAttr

def checkout_count_features(train_userAttr):
    train_userAttr['open_per_checkout'] = train_userAttr['open_PC_0'] / train_userAttr['checkout_PC_0']
    train_userAttr['checkout_count_per10day_last_60_days'] = train_userAttr['checkout_count_last_60_days'] / 6
    train_userAttr['checkout_count_per10day_last_30_days'] = train_userAttr['checkout_count_last_30_days'] / 3
    train_userAttr['checkout_count_trend_60-30'] = train_userAttr['checkout_count_per10day_last_30_days'] - train_userAttr['checkout_count_per10day_last_60_days']
    train_userAttr['checkout_count_trend_30-10'] = train_userAttr['checkout_count_last_10_days'] - train_userAttr['checkout_count_per10day_last_30_days']
    return train_userAttr

train_userAttr = pca_feature_transform(train_userAttr, 'open', n_components=1)
train_userAttr = pca_feature_transform(train_userAttr, 'login', n_components=1)
train_userAttr = pca_feature_transform(train_userAttr, 'checkout', n_components=1)
test_userAttr = pca_feature_transform(test_userAttr, 'open', n_components=1)
test_userAttr = pca_feature_transform(test_userAttr, 'login', n_components=1)
test_userAttr = pca_feature_transform(test_userAttr, 'checkout', n_components=1)
train_userAttr = open_count_features(train_userAttr)
test_userAttr = open_count_features(test_userAttr)
train_userAttr = login_count_features(train_userAttr)
test_userAttr = login_count_features(test_userAttr)
train_userAttr = checkout_count_features(train_userAttr)
test_userAttr = checkout_count_features(test_userAttr)

for s in ['open','login','checkout']:
    train_userAttr = train_userAttr.drop([f'last_{s}_day'], axis=1)
    test_userAttr = test_userAttr.drop([f'last_{s}_day'], axis=1)

    for t in ['10','30','60']:
        if t != '10':
            train_userAttr = train_userAttr.drop([f'{s}_count_per10day_last_{t}_days'], axis=1)
            test_userAttr = test_userAttr.drop([f'{s}_count_per10day_last_{t}_days'], axis=1)
            train_userAttr = train_userAttr.drop([f'{s}_count_last_{t}_days'], axis=1)
            test_userAttr = test_userAttr.drop([f'{s}_count_last_{t}_days'], axis=1)
        else:
            train_userAttr = train_userAttr.drop([f'{s}_count_last_{t}_days'], axis=1)
            test_userAttr = test_userAttr.drop([f'{s}_count_last_{t}_days'], axis=1)            

train_userAttr.head()
test_userAttr.head()


open [0.96667058]
login [0.98358528]
checkout [0.96459218]
open [0.97026378]
login [0.9836373]
checkout [0.96057233]


,user_id,subject_line_length,open_flag,row_id,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age_type_<30,age_type_>=30,attr_1_0,attr_1_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_type_high_domain,domain_type_med_domain,open_PC_0,login_PC_0,checkout_PC_0,open_count_trend_60-30,open_count_trend_30-10,open_per_login,login_count_trend_60-30,login_count_trend_30-10,open_per_checkout,checkout_count_trend_60-30,checkout_count_trend_30-10
0,43,44,0,0,0,0,0,16,2,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,-1.400586,52.491066,4.888114,0.000000,-0.666667,-0.026682,-2.166667,-2.333333,-0.286529,0.000000,-1.666667
1,102,44,1,1,0,0,0,16,2,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,13.491641,47.755599,-2.059964,0.166667,-1.000000,0.282514,1.000000,2.000000,-6.549455,-0.333333,0.666667
2,177,49,0,2,0,0,0,16,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.451121,83.660475,26.964996,-0.666667,-1.333333,0.077111,3.166667,1.000000,0.239241,1.833333,-1.333333
3,184,49,0,3,0,0,0,16,2,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,-4.949294,16.611117,0.613821,-0.166667,0.000000,-0.297951,-3.833333,1.333333,-8.063090,0.000000,0.000000
4,221,49,0,4,0,0,0,16,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-5.809894,-48.819961,-6.166705,0.000000,0.000000,0.119007,0.833333,0.333333,0.942139,0.000000,0.000000


,user_id,subject_line_length,row_id,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age_type_<30,age_type_>=30,attr_1_0,attr_1_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_type_high_domain,domain_type_med_domain,open_PC_0,login_PC_0,checkout_PC_0,open_count_trend_60-30,open_count_trend_30-10,open_per_login,login_count_trend_60-30,login_count_trend_30-10,open_per_checkout,checkout_count_trend_60-30,checkout_count_trend_30-10
0,0,35,0,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-0.739284,76.780259,11.518758,0.333333,1.000000,-0.009629,-11.000000,-1.333333,-0.064181,-1.000000,-2.000000
1,130,35,1,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-4.427142,-50.401652,-6.843296,0.166667,0.666667,0.087837,0.833333,3.333333,0.646931,0.000000,0.000000
2,150,25,2,0,0,0,3,2,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,-5.902257,-13.853989,-3.817512,0.000000,0.000000,0.426033,0.000000,6.666667,1.546100,0.333333,1.333333
3,181,36,3,0,0,0,3,2,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,-5.902257,150.538057,0.202209,0.000000,0.000000,-0.039208,7.833333,6.333333,-29.188931,0.833333,0.333333
4,192,23,4,0,0,0,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-5.902257,-17.527147,-5.122861,0.000000,0.000000,0.336749,-2.500000,0.000000,1.152141,-0.333333,0.000000


#### Polynomial Features 
- https://heartbeat.fritz.ai/a-practical-guide-to-feature-engineering-in-python-8326e40747c8

'subject_line_length', 'last_open_day', 'last_login_day', 'last_checkout_day', 'open_count_last_10_days', 
                     'open_count_last_30_days', 'open_count_last_60_days', 'login_count_last_10_days', 
                     'login_count_last_30_days', 'login_count_last_60_days', 'checkout_count_last_10_days', 
                     'checkout_count_last_30_days', 'checkout_count_last_60_days', 'open_count_per10day_last_60_days',
                     'login_count_per10day_last_60_days', 'checkout_count_per10day_last_60_days', 'open_count_per10day_last_30_days', 'login_count_per10day_last_30_days', 'checkout_count_per10day_last_30_days'
 
> - `open_count_last_10_days` * `open_count_last_10_days`
> - `open_count_last_10_days` * `open_count_last_30_days`
> - `open_count_last_10_days` * `open_count_last_60_days`



In [169]:
# train_target.iloc[:, np.r_[np.where((RFC.feature_importances_ < 0.05)&(RFC.feature_importances_ > 0.005))]].columns
# train_target.iloc[:, np.r_[np.where(RFC.feature_importances_ < 0.005)]].columns
np.where((RFC.feature_importances_ < 0.05)&(RFC.feature_importances_ > 0.005))
np.where((RFC.feature_importances_ < 0.005))

# train_userAttr = train_userAttr.drop(train_target.iloc[:, np.r_[np.where(RFC.feature_importances_ < 0.005)]].columns, axis=1)
# test_userAttr = test_userAttr.drop(train_target.iloc[:, np.r_[np.where(RFC.feature_importances_ < 0.005)]].columns, axis=1)


(array([ 0,  4,  5,  6,  7,  8,  9, 10, 12, 20, 30, 31, 35, 36, 38, 39],
       dtype=int64),)

(array([ 1,  2,  3, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26,
        27, 28], dtype=int64),)

In [719]:
def polynomial_feature(train_userAttr, selected_col, numeric_only=True):
    if numeric_only == True:
        selected_col = selected_col
#         selected_col = ['subject_line_length','day_of_week']
    else:
        print('crossing all features...')
        selected_col = [c for c in train_userAttr.columns if c not in ['open_flag','user_id','row_id']]
        
    v = poly.fit_transform(train_userAttr[selected_col])
    left_col = [c for c in train_userAttr if c not in selected_col]
    return v, left_col


selected_cols = [['subject_line_length','day_of_week'], 
                 ['open_count_last_10_days','open_count_last_30_days','open_count_last_60_days'], 
                 ['last_login_day', 'last_checkout_day', 'login_count_last_10_days',
       'login_count_last_30_days', 'login_count_last_60_days',
       'checkout_count_last_30_days', 'checkout_count_last_60_days',
       'country_code_1', 'country_code_2', 'country_code_3', 'country_code_4',
       'country_code_5', 'age', 'domain_@gmail.com',
       'login_count_per10day_last_60_days',
       'login_count_per10day_last_30_days', 'login_count_trend_60-30',
       'login_count_trend_30-10', 'checkout_count_per10day_last_60_days',
       'checkout_count_per10day_last_30_days', 'checkout_count_trend_60-30',
       'checkout_count_trend_30-10']]
for _set in selected_cols:
    
    poly_features, left_col = polynomial_feature(train_userAttr, selected_col=_set, numeric_only=True)
    poly_features = pd.DataFrame(poly_features, columns=['poly_features_' + str(x) for x in range(0, poly_features.shape[1])])
    train_userAttr = pd.concat([train_userAttr[left_col], poly_features], axis=1)


    poly_features, left_col = polynomial_feature(test_userAttr, selected_col=_set, numeric_only=True)
    poly_features = pd.DataFrame(poly_features, columns=['poly_features_' + str(x) for x in range(0, poly_features.shape[1])])
    test_userAttr = pd.concat([test_userAttr[left_col], poly_features], axis=1)

train_userAttr.shape
train_userAttr.head()
test_userAttr.shape
test_userAttr.head()


(73539, 331)

,user_id,last_open_day,checkout_count_last_10_days,open_flag,row_id,country_code_6,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,poly_features_0,poly_features_0,poly_features_1,poly_features_1,poly_features_2,poly_features_2,poly_features_3,poly_features_3,poly_features_4,poly_features_4,poly_features_5,poly_features_5,poly_features_0,poly_features_0,poly_features_1,poly_features_1,poly_features_2,poly_features_2,poly_features_3,poly_features_3,poly_features_4,poly_features_4,poly_features_5,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9,poly_features_10,poly_features_11,poly_features_12,poly_features_13,poly_features_14,poly_features_15,poly_features_16,poly_features_17,poly_features_18,poly_features_19,poly_features_20,poly_features_21,poly_features_22,poly_features_23,poly_features_24,poly_features_25,poly_features_26,poly_features_27,poly_features_28,poly_features_29,poly_features_30,poly_features_31,poly_features_32,poly_features_33,poly_features_34,poly_features_35,poly_features_36,poly_features_37,poly_features_38,poly_features_39,poly_features_40,poly_features_41,poly_features_42,poly_features_43,poly_features_44,poly_features_45,poly_features_46,poly_features_47,poly_features_48,poly_features_49,poly_features_50,poly_features_51,poly_features_52,poly_features_53,poly_features_54,poly_features_55,poly_features_56,poly_features_57,poly_features_58,poly_features_59,poly_features_60,poly_features_61,poly_features_62,poly_features_63,poly_features_64,poly_features_65,poly_features_66,poly_features_67,poly_features_68,poly_features_69,poly_features_70,poly_features_71,poly_features_72,poly_features_73,poly_features_74,poly_features_75,poly_features_76,poly_features_77,poly_features_78,poly_features_79,poly_features_80,poly_features_81,poly_features_82,poly_features_83,poly_features_84,poly_features_85,poly_features_86,poly_features_87,poly_features_88,poly_features_89,poly_features_90,poly_features_91,poly_features_92,poly_features_93,poly_features_94,poly_features_95,poly_features_96,poly_features_97,poly_features_98,poly_features_99,poly_features_100,poly_features_101,poly_features_102,poly_features_103,poly_features_104,poly_features_105,poly_features_106,poly_features_107,poly_features_108,poly_features_109,poly_features_110,poly_features_111,poly_features_112,poly_features_113,poly_features_114,poly_features_115,poly_features_116,poly_features_117,poly_features_118,poly_features_119,poly_features_120,poly_features_121,poly_features_122,poly_features_123,poly_features_124,poly_features_125,poly_features_126,poly_features_127,poly_features_128,poly_features_129,poly_features_130,poly_features_131,poly_features_132,poly_features_133,poly_features_134,poly_features_135,poly_features_136,poly_features_137,poly_features_138,poly_features_139,poly_features_140,poly_features_141,poly_features_142,poly_features_143,poly_features_144,poly_features_145,poly_features_146,poly_features_147,poly_features_148,poly_features_149,poly_features_150,poly_features_151,poly_features_152,poly_features_153,poly_features_154,poly_features_155,poly_features_156,poly_features_157,poly_features_158,poly_features_159,poly_features_160,poly_features_161,poly_features_162,poly_features_163,poly_features_164,poly_features_165,poly_features_166,poly_features_167,poly_features_168,poly_features_169,poly_features_170,poly_features_171,poly_features_172,poly_features_173,poly_features_174,poly_features_175,poly_features_176,poly_features_177,poly_features_178,poly_fe

(55970, 330)

,user_id,last_open_day,checkout_count_last_10_days,row_id,country_code_6,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,poly_features_0,poly_features_0,poly_features_1,poly_features_1,poly_features_2,poly_features_2,poly_features_3,poly_features_3,poly_features_4,poly_features_4,poly_features_5,poly_features_5,poly_features_0,poly_features_0,poly_features_1,poly_features_1,poly_features_2,poly_features_2,poly_features_3,poly_features_3,poly_features_4,poly_features_4,poly_features_5,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9,poly_features_10,poly_features_11,poly_features_12,poly_features_13,poly_features_14,poly_features_15,poly_features_16,poly_features_17,poly_features_18,poly_features_19,poly_features_20,poly_features_21,poly_features_22,poly_features_23,poly_features_24,poly_features_25,poly_features_26,poly_features_27,poly_features_28,poly_features_29,poly_features_30,poly_features_31,poly_features_32,poly_features_33,poly_features_34,poly_features_35,poly_features_36,poly_features_37,poly_features_38,poly_features_39,poly_features_40,poly_features_41,poly_features_42,poly_features_43,poly_features_44,poly_features_45,poly_features_46,poly_features_47,poly_features_48,poly_features_49,poly_features_50,poly_features_51,poly_features_52,poly_features_53,poly_features_54,poly_features_55,poly_features_56,poly_features_57,poly_features_58,poly_features_59,poly_features_60,poly_features_61,poly_features_62,poly_features_63,poly_features_64,poly_features_65,poly_features_66,poly_features_67,poly_features_68,poly_features_69,poly_features_70,poly_features_71,poly_features_72,poly_features_73,poly_features_74,poly_features_75,poly_features_76,poly_features_77,poly_features_78,poly_features_79,poly_features_80,poly_features_81,poly_features_82,poly_features_83,poly_features_84,poly_features_85,poly_features_86,poly_features_87,poly_features_88,poly_features_89,poly_features_90,poly_features_91,poly_features_92,poly_features_93,poly_features_94,poly_features_95,poly_features_96,poly_features_97,poly_features_98,poly_features_99,poly_features_100,poly_features_101,poly_features_102,poly_features_103,poly_features_104,poly_features_105,poly_features_106,poly_features_107,poly_features_108,poly_features_109,poly_features_110,poly_features_111,poly_features_112,poly_features_113,poly_features_114,poly_features_115,poly_features_116,poly_features_117,poly_features_118,poly_features_119,poly_features_120,poly_features_121,poly_features_122,poly_features_123,poly_features_124,poly_features_125,poly_features_126,poly_features_127,poly_features_128,poly_features_129,poly_features_130,poly_features_131,poly_features_132,poly_features_133,poly_features_134,poly_features_135,poly_features_136,poly_features_137,poly_features_138,poly_features_139,poly_features_140,poly_features_141,poly_features_142,poly_features_143,poly_features_144,poly_features_145,poly_features_146,poly_features_147,poly_features_148,poly_features_149,poly_features_150,poly_features_151,poly_features_152,poly_features_153,poly_features_154,poly_features_155,poly_features_156,poly_features_157,poly_features_158,poly_features_159,poly_features_160,poly_features_161,poly_features_162,poly_features_163,poly_features_164,poly_features_165,poly_features_166,poly_features_167,poly_features_168,poly_features_169,poly_features_170,poly_features_171,poly_features_172,poly_features_173,poly_features_174,poly_features_175,poly_features_176,poly_features_177,poly_features_178,poly_features_179

In [ ]:
### compare filter of label amounts
for c in train_userAttr.columns[1:]:
    if 'attr' in c:
        train_userAttr[train_userAttr['open_flag']==1].groupby([c]).size()
        train_userAttr[train_userAttr['open_flag']==0].groupby([c]).size()



### EDA findings
- `subject_line_length`: 10, 33, >50 開信率較低、20-30, 35-45 開信率較高
- `last_open_day`: 距離越近開過信的人，越可能開下一封
- `open_count_last_10_days`: 0-1次開信率較低、大於2次開信率相對越高
- `open_count_last_30_days`: 0-1次開信率較低、大於2次開信率相對越高
- `open_count_last_60_days`: 0-1次開信率較低、大於2次開信率相對越高
- `checkout_count_last_10_days`: 0次開信率較高、大於1次開信率相對越低
- `checkout_count_last_30_days`: 0次開信率較高、大於1次開信率相對越低
- `checkout_count_last_60_days`: 0次開信率較高、大於1次開信率相對越低


In [421]:
# 1: orange
train_userAttr.iloc[:,55]
# train_userAttr.iloc[:, 3:] = rs.fit_transform(train_userAttr.iloc[:, 3:].values)
# for i in range(235, 245): #20 #90-92
#     sns.pairplot(train_userAttr.iloc[:, np.r_[1, i]], hue='open_flag', height=5)
#     plt.show()

# for i in range(16, len(train_userAttr.columns)):
#     sns.pairplot(train_userAttr.iloc[:, np.r_[i, 14]], hue='open_flag', size=5)
#     plt.show()


0        0.056202
1        0.056202
2        0.356589
3        0.356589
4        0.356589
           ...   
73534   -0.211886
73535   -0.261628
73536   -0.532946
73537   -0.261628
73538   -0.211886
Name: poly_features_20, Length: 73539, dtype: float64

In [415]:
# sns.pairplot(train.iloc[:, np.r_[7:10, 16]], hue='open_flag')
# sns.pairplot(train.iloc[:, np.r_[10:13, 16]], hue='open_flag')
# sns.pairplot(train.iloc[:, np.r_[13:17]], hue='open_flag')

# sns.pairplot(train_userAttr[['age','open_flag']], hue='open_flag')
# train_userAttr.groupby(['open_flag','age']).size()[:50]


In [ ]:
user_attr.groupby(['attr_1']).size()
user_attr.groupby(['attr_2']).size()
user_attr.groupby(['attr_3']).size()
user_attr.groupby(['age']).size()
user_attr.groupby(['domain']).size()
test_userAttr[test_userAttr['last_checkout_day']!='Never checkout'].last_checkout_day.max()


In [297]:
# sns.distplot(train_userAttr[train_userAttr['last_open_day']<=808].last_open_day)
# sns.distplot(train_userAttr[train_userAttr['last_login_day']<=200].last_login_day, kde=False, rug=True)
# sns.distplot(train_userAttr[train_userAttr['last_checkout_day']<=200].last_checkout_day, kde=False, rug=True)
# train_userAttr.groupby(['open_flag']).size()

# sns.boxplot(user_attr.age)
# boxplot_stats(user_attr['age']).pop(0)['fliers']
# outliers = [y for stat in boxplot_stats(user_attr.age) for y in stat['fliers']]
# print(outliers)
# sns.swarmplot(user_attr.age)


#### train data


In [4]:
# train_userAttr.shape
train_target = train_userAttr.iloc[:, np.r_[1, 4:36]]
# train_target = train_target.iloc[:, np.r_[np.where(RFC.feature_importances_ > 0.01)]]
# train_target = train_userAttr.iloc[:, 3:train_userAttr.shape[1]]
train_target.shape
train_target.head()


(73539, 33)

,subject_line_length,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age_type_<30,age_type_>=30,attr_1_0,attr_1_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_type_high_domain,domain_type_med_domain,open_PC_0,login_PC_0,checkout_PC_0,open_count_trend_60-30,open_count_trend_30-10,open_per_login,login_count_trend_60-30,login_count_trend_30-10,open_per_checkout,checkout_count_trend_60-30,checkout_count_trend_30-10
0,44,0,0,0,16,2,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,-1.400586,52.491066,4.888114,0.000000,-0.666667,-0.026682,-2.166667,-2.333333,-0.286529,0.000000,-1.666667
1,44,0,0,0,16,2,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,13.491641,47.755599,-2.059964,0.166667,-1.000000,0.282514,1.000000,2.000000,-6.549455,-0.333333,0.666667
2,49,0,0,0,16,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.451121,83.660475,26.964996,-0.666667,-1.333333,0.077111,3.166667,1.000000,0.239241,1.833333,-1.333333
3,49,0,0,0,16,2,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,-4.949294,16.611117,0.613821,-0.166667,0.000000,-0.297951,-3.833333,1.333333,-8.063090,0.000000,0.000000
4,49,0,0,0,16,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-5.809894,-48.819961,-6.166705,0.000000,0.000000,0.119007,0.833333,0.333333,0.942139,0.000000,0.000000


#### PCA features
- Local MCC 提升，但Kaggle MCC 下降約0.04

In [5]:
def pca_feature(train_target, feature_name, n_components=10):
    pca = PCA(n_components=n_components)
    if feature_name == 'open':
        _PCA = pca.fit_transform(train_target.iloc[:, np.r_[1, 4:7]].values)
        
    elif feature_name == 'login':
        _PCA = pca.fit_transform(train_target.iloc[:, np.r_[2, 7:10]].values)
        
    elif feature_name == 'checkout':
        _PCA = pca.fit_transform(train_target.iloc[:, np.r_[3, 10:13]].values)
        
    elif feature_name == 'country':
        _PCA = pca.fit_transform(train_target.iloc[:, np.r_[14:20]].values)
        
    elif feature_name == 'attr':
        _PCA = pca.fit_transform(train_target.iloc[:, np.r_[21:29]].values)
    
    elif feature_name == 'domain':
        _PCA = pca.fit_transform(train_target.iloc[:, np.r_[29:42]].values)
    
    else:
        print('must choose from: open, login, checkout, country, attr, and domain')
        return 
    
    _PCA = pd.DataFrame(_PCA, columns=[f'{feature_name}_PC_' + str(x) for x in range(0, n_components)])
    print(pca.explained_variance_ratio_)
    
    return _PCA


# pca = PCA(n_components=2)
# open_PCA = pca.fit_transform(train_target.iloc[:, np.r_[1, 4:7]].values)
# open_PCA = pd.DataFrame(open_PCA, columns=['open_PC_' + str(x) for x in range(0,2)])
# print(pca.explained_variance_ratio_)
# open_PCA.head()

# pca = PCA(n_components=2)
# login_PCA = pca.fit_transform(train_target.iloc[:, np.r_[2, 7:10]].values)
# login_PCA = pd.DataFrame(login_PCA, columns=['login_PCA_' + str(x) for x in range(0,2)])
# print(pca.explained_variance_ratio_)
# login_PCA.head()

# pca = PCA(n_components=2)
# checkout_PCA = pca.fit_transform(train_target.iloc[:, np.r_[3, 10:13]].values)
# checkout_PCA = pd.DataFrame(checkout_PCA, columns=['checkout_PCA_' + str(x) for x in range(0,2)])
# print(pca.explained_variance_ratio_)
# checkout_PCA.head()

# pca = PCA(n_components=4)
# country_PCA = pca.fit_transform(train_target.iloc[:, np.r_[14:20]].values)
# country_PCA = pd.DataFrame(country_PCA, columns=['country_PCA_' + str(x) for x in range(0,4)])
# print(pca.explained_variance_ratio_)
# country_PCA.head()

# pca = PCA(n_components=2)
# attr_PCA = pca.fit_transform(train_target.iloc[:, np.r_[21:29]].values)
# attr_PCA = pd.DataFrame(attr_PCA, columns=['attr_PCA_' + str(x) for x in range(0,2)])
# print(pca.explained_variance_ratio_)
# attr_PCA.head()

# pca = PCA(n_components=2)
# domain_PCA = pca.fit_transform(train_target.iloc[:, np.r_[29:42]].values)
# domain_PCA = pd.DataFrame(domain_PCA, columns=['domain_PCA_' + str(x) for x in range(0,2)])
# print(pca.explained_variance_ratio_)
# domain_PCA.head()

# open_PCA = pca_feature(train_target, feature_name='open', n_components=2)
# login_PCA = pca_feature(train_target, feature_name='login', n_components=2)
# checkout_PCA = pca_feature(train_target, feature_name='checkout', n_components=2)
# country_PCA = pca_feature(train_target, feature_name='country', n_components=4)
# attr_PCA = pca_feature(train_target, feature_name='attr', n_components=2)
# domain_PCA = pca_feature(train_target, feature_name='domain', n_components=2)

# train_target = pd.concat([train_target.iloc[:, np.r_[0, 13, 20, 42:51]], open_PCA], axis=1)
# train_target = pd.concat([train_target, login_PCA], axis=1)
# train_target = pd.concat([train_target, checkout_PCA], axis=1)
# train_target = pd.concat([train_target, country_PCA], axis=1)
# train_target = pd.concat([train_target, attr_PCA], axis=1)
# train_target = pd.concat([train_target, domain_PCA], axis=1)
# train_target.shape
# train_target.head()


In [215]:
train_target.to_csv('train_target_with_PCA_features.csv', index=False)

#### data split

In [5]:
print(train_target.shape)
print(Y.shape)

train_x, val_x, train_y, val_y = train_test_split(train_target, Y, test_size=0.35, random_state=832, stratify=Y)
print('train sets: ', train_x.shape[0], '\nval sets: ', val_x.shape[0])


(73539, 33)
(73539,)
train sets:  47800 
val sets:  25739


In [6]:
test_target = test_userAttr.iloc[:, np.r_[1, 3:35]]
# test_target = test_target.iloc[:, np.r_[np.where(RFC.feature_importances_ > 0.01)]]
# test_target = test_userAttr.iloc[:, 2:test_userAttr.shape[1]]
test_target.head()
test_target.shape


,subject_line_length,isNeverOpen,isNeverLogin,isNeverCheckout,day,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age_type_<30,age_type_>=30,attr_1_0,attr_1_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_type_high_domain,domain_type_med_domain,open_PC_0,login_PC_0,checkout_PC_0,open_count_trend_60-30,open_count_trend_30-10,open_per_login,login_count_trend_60-30,login_count_trend_30-10,open_per_checkout,checkout_count_trend_60-30,checkout_count_trend_30-10
0,35,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-0.739284,76.780259,11.518758,0.333333,1.000000,-0.009629,-11.000000,-1.333333,-0.064181,-1.000000,-2.000000
1,35,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-4.427142,-50.401652,-6.843296,0.166667,0.666667,0.087837,0.833333,3.333333,0.646931,0.000000,0.000000
2,25,0,0,0,3,2,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,-5.902257,-13.853989,-3.817512,0.000000,0.000000,0.426033,0.000000,6.666667,1.546100,0.333333,1.333333
3,36,0,0,0,3,2,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,-5.902257,150.538057,0.202209,0.000000,0.000000,-0.039208,7.833333,6.333333,-29.188931,0.833333,0.333333
4,23,0,0,0,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-5.902257,-17.527147,-5.122861,0.000000,0.000000,0.336749,-2.500000,0.000000,1.152141,-0.333333,0.000000


(55970, 33)

#### PCA features for testset

In [515]:
# open_PCA = pca_feature(train_target=test_target, feature_name='open', n_components=2)
# login_PCA = pca_feature(train_target=test_target, feature_name='login', n_components=2)
# checkout_PCA = pca_feature(train_target=test_target, feature_name='checkout', n_components=2)
# country_PCA = pca_feature(train_target=test_target, feature_name='country', n_components=4)
# attr_PCA = pca_feature(train_target=test_target, feature_name='attr', n_components=2)
# domain_PCA = pca_feature(train_target=test_target, feature_name='domain', n_components=2)

# test_target = pd.concat([test_target.iloc[:, np.r_[0, 13, 20, 42:51]], open_PCA], axis=1)
# test_target = pd.concat([test_target, login_PCA], axis=1)
# test_target = pd.concat([test_target, checkout_PCA], axis=1)
# test_target = pd.concat([test_target, country_PCA], axis=1)
# test_target = pd.concat([test_target, attr_PCA], axis=1)
# test_target = pd.concat([test_target, domain_PCA], axis=1)
# test_target.shape
# test_target.head()


In [223]:
test_target.to_csv('test_target_with_PCA_features.csv', index=False)

#### scaling after split


In [7]:
# for c in train_userAttr.columns:
#     if c in ['subject_line_length', 'last_open_day', 'last_login_day', 'last_checkout_day', 'open_count_last_10_days', 
#              'open_count_last_30_days', 'open_count_last_60_days', 'login_count_last_10_days', 
#              'login_count_last_30_days', 'login_count_last_60_days', 'checkout_count_last_10_days', 
#              'checkout_count_last_30_days', 'checkout_count_last_60_days', 'age', 'day_of_week', 'open_count_per10day_last_60_days',
#              'login_count_per10day_last_60_days', 'checkout_count_per10day_last_60_days', 'open_count_per10day_last_30_days',
#              'login_count_per10day_last_30_days', 'checkout_count_per10day_last_30_days', 'open_count_trend_60-30',
#              'login_count_trend_60-30', 'checkout_count_trend_60-30', 'open_count_trend_30-10',
#              'login_count_trend_30-10', 'checkout_count_trend_30-10']:
#         train_x[c] = sd.fit(train_x[c].values.reshape(-1,1))
#         train_x[c] = sd.transform(train_x[c].values.reshape(-1,1))
#         val_x[c] = sd.transform(val_x[c].values.reshape(-1,1))

sd.fit(train_x)
train_x = sd.transform(train_x)
train_x.shape
val_x = sd.transform(val_x)
val_x.shape
test_x = sd.transform(test_target)
test_x.shape


StandardScaler()

(47800, 33)

(25739, 33)

(55970, 33)

#### model tuning
- RF (better) 0.491

In [225]:
produce_date = 200808
model = 'RF'
n_estimatorss = [300, 250, 200, 150, 100, 80]  #200
max_depths = [30, 25, 20] #20
min_samples_leafs = [7, 9, 11, 13, 15, 17]  #7
min_samples_splits = [7, 9, 11, 13, 15, 17]  #17
class_weights = [{0: 1., 1: 4.}, None]  #None
oob_score = True
feature_desc = 'EDA_reference_featureOpt_standardlizedScaled'


In [8]:
# for class_weight in class_weights:
n_estimators = 200
max_depth = 20
min_samples_leaf = 7
min_samples_split = 17
class_weight = None
oob_score = True
model_desc = f'n_estimators_{n_estimators}-max_depth_{max_depth}-min_samples_leaf_{min_samples_leaf}-min_samples_split_{min_samples_split}-class_weight_{str(class_weight)}-oob_score_{oob_score}'
print(model_desc)
RFC = RandomForestClassifier(n_estimators=n_estimators, 
                             oob_score=oob_score, 
                             max_depth=max_depth, 
                             min_samples_leaf=min_samples_leaf, 
                             min_samples_split=min_samples_split, 
                             class_weight=class_weight, 
                             random_state=39)
RFC.fit(train_x, train_y)
print(RFC.oob_score_)
# print(RFC.feature_importances_.tolist().index(RFC.feature_importances_.max()))
# print(RFC.feature_importances_)
y_pred = RFC.predict(val_x)
print(classification_report(y_true=val_y, y_pred=y_pred))
print(confusion_matrix(y_true=val_y, y_pred=y_pred))
print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


n_estimators_200-max_depth_20-min_samples_leaf_7-min_samples_split_17-class_weight_None-oob_score_True


RandomForestClassifier(max_depth=20, min_samples_leaf=7, min_samples_split=17,
                       n_estimators=200, oob_score=True, random_state=39)

0.8852719665271966
              precision    recall  f1-score   support

           0       0.90      0.97      0.94     21729
           1       0.74      0.44      0.56      4010

    accuracy                           0.89     25739
   macro avg       0.82      0.71      0.75     25739
weighted avg       0.88      0.89      0.88     25739

[[21117   612]
 [ 2233  1777]]
AUC score:  0.707488509845089
MCC score:  0.5186383014364572


In [99]:
# np.where((RFC.feature_importances_ < 0.05)&(RFC.feature_importances_ > 0.005))
# np.where((RFC.feature_importances_ < 0.005))

In [230]:
# y_pred = RFC.predict(val_x)
# print(classification_report(y_true=val_y, y_pred=y_pred))
# print(confusion_matrix(y_true=val_y, y_pred=y_pred))
# print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
# print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
# model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


In [ ]:
# # Extract single tree
# index = 0
# estimator = RFC.estimators_[index]

# # Export as png file
# fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (4,4), dpi=800)
# tree.plot_tree(estimator, feature_names = train_target.columns, class_names = ['not open', 'open'],
#                 rounded = True, proportion = False, 
#                 precision = 2, filled = True)

# fig.savefig(f'rf_individualtree_{index}.png')

# # Convert to png using system command (requires Graphviz)
# # os.system('dot -Tpng tree.dot -o tree.png')
# # call(['dot', '-Tpng', f'tree_{produce_date}_{model}_{model_desc}_{feature_desc}.dot', '-o', f'tree_{produce_date}_{model}_{model_desc}_{feature_desc}.png', '-Gdpi=600'])

# # Display in jupyter notebook
# # Image(filename = 'tree.png')


- GradientBoostingClassifer 0.480

In [17]:
produce_date = 200806
model = 'GBC'
n_estimatorss = [300, 250, 200, 150, 100, 80]
learning_rate = [0.1, 0.15, 0.2]
max_depths = [20, 15, 12, 10, 8]
min_samples_leafs = [70,60,50]
min_samples_splits = [70,60,50]
class_weights = [{0: 1., 1: 4}, None]
feature_desc = 'EDA_reference_featureOpt_standardlizedScaled'


In [19]:
for min_samples_split in min_samples_splits:
    n_estimators = 80
    max_depth = 10
    learning_rate = 0.1
    min_samples_leaf = 60
#     min_samples_split = 50
    class_weight = None
    model_desc = f'n_estimators_{n_estimators}-max_depth_{max_depth}-min_samples_leaf_{min_samples_leaf}-min_samples_split_{min_samples_split}-class_weight_{str(class_weight)}'
    print(model_desc)
    gbc = GradientBoostingClassifier(n_estimators=n_estimators,
                                     learning_rate=learning_rate,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     max_depth=max_depth,
                                     max_features='sqrt',
                                     subsample=0.7,
                                     random_state=99)
    gbc.fit(train_x, train_y)
    y_pred = gbc.predict(val_x)
    print(classification_report(y_true=val_y, y_pred=y_pred))
    print(confusion_matrix(y_true=val_y, y_pred=y_pred))
    print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
    print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
#     model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


In [233]:
# y_pred = gbc.predict(val_x)
# print(classification_report(y_true=val_y, y_pred=y_pred))
# print(confusion_matrix(y_true=val_y, y_pred=y_pred))
# print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
# print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
# model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


- XGBoost
https://www.kaggle.com/babatee/intro-xgboost-classification

In [234]:
produce_date = 200808
model = 'XGB'
n_estimatorss = [305, 255, 205, 155, 105, 85]
max_depths = [25, 20, 15, 12, 8]
learning_rates = [0.25,0.2,0.15,0.1]
subsamples = [0.85]
class_weights = [{0: 1., 1: 4.5}, None]
feature_desc = 'EDA_reference_featureOpt_standardlizedScaled'


In [235]:
for n_estimators in n_estimatorss:
    #     n_estimators = 300
    max_depth = 20
    learning_rate = 0.15
    subsample = 0.85
    class_weight = None
    model_desc = f'n_estimators_{n_estimators}-max_depth_{max_depth}-learning_rate_{learning_rate}-subsample_{subsample}'
    print(model_desc)
    XGB = xgb.XGBClassifier(n_estimators=n_estimators, 
                            max_depth=max_depth, 
                            learning_rate=learning_rate, 
                            subsample=subsample,
                            class_weight=class_weight,
                            random_state=34)
    XGB.fit(train_x, train_y)
    y_pred = gbc.predict(val_x)
    print(classification_report(y_true=val_y, y_pred=y_pred))
    print(confusion_matrix(y_true=val_y, y_pred=y_pred))
    print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
    print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
#     model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


n_estimators_305-max_depth_20-learning_rate_0.15-subsample_0.85


XGBClassifier(base_score=0.5, booster='gbtree', class_weight=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.15, max_delta_step=0,
              max_depth=20, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=305, n_jobs=0,
              num_parallel_tree=1, random_state=34, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.85, tree_method='exact',
              validate_parameters=1, verbosity=None)

              precision    recall  f1-score   support

           0       0.91      0.96      0.94     21729
           1       0.70      0.48      0.57      4010

    accuracy                           0.89     25739
   macro avg       0.81      0.72      0.75     25739
weighted avg       0.88      0.89      0.88     25739

[[20915   814]
 [ 2090  1920]]
AUC score:  0.7206707677398615
MCC score:  0.51946523034598
n_estimators_255-max_depth_20-learning_rate_0.15-subsample_0.85


XGBClassifier(base_score=0.5, booster='gbtree', class_weight=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.15, max_delta_step=0,
              max_depth=20, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=255, n_jobs=0,
              num_parallel_tree=1, random_state=34, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.85, tree_method='exact',
              validate_parameters=1, verbosity=None)

              precision    recall  f1-score   support

           0       0.91      0.96      0.94     21729
           1       0.70      0.48      0.57      4010

    accuracy                           0.89     25739
   macro avg       0.81      0.72      0.75     25739
weighted avg       0.88      0.89      0.88     25739

[[20915   814]
 [ 2090  1920]]
AUC score:  0.7206707677398615
MCC score:  0.51946523034598
n_estimators_205-max_depth_20-learning_rate_0.15-subsample_0.85


XGBClassifier(base_score=0.5, booster='gbtree', class_weight=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.15, max_delta_step=0,
              max_depth=20, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=205, n_jobs=0,
              num_parallel_tree=1, random_state=34, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.85, tree_method='exact',
              validate_parameters=1, verbosity=None)

              precision    recall  f1-score   support

           0       0.91      0.96      0.94     21729
           1       0.70      0.48      0.57      4010

    accuracy                           0.89     25739
   macro avg       0.81      0.72      0.75     25739
weighted avg       0.88      0.89      0.88     25739

[[20915   814]
 [ 2090  1920]]
AUC score:  0.7206707677398615
MCC score:  0.51946523034598
n_estimators_155-max_depth_20-learning_rate_0.15-subsample_0.85


KeyboardInterrupt: 

In [ ]:
# y_pred = XGB.predict(val_x)
# print(classification_report(y_true=val_y, y_pred=y_pred))
# print(confusion_matrix(y_true=val_y, y_pred=y_pred))
# print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
# print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
# model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


- lightGBM
    - https://www.programcreek.com/python/example/88793/lightgbm.LGBMClassifier

In [205]:
produce_date = 200808
model = 'LGBM'
n_estimatorss = [305, 255, 205, 155, 105, 85]  #105
max_depths = [30, 25, 20, 15, 12, 8, 6]  #6
learning_rates = [0.25, 0.2, 0.1, 0.05]  #0.2
num_leavess = [70, 50]  #50
bagging_fractions = [0.6, 0.75, 0.85]
feature_desc = 'EDA_reference_featureOpt_standardlizedScaled'


In [208]:
# for learning_rate in learning_rates:
n_estimators = 105
max_depth = 6
learning_rate = 0.2
num_leaves = 50
bagging_fraction = 0.85
model_desc = f'n_estimators_{n_estimators}-max_depth_{max_depth}-num_leaves_{num_leaves}-learning_rate_{learning_rate}-bagging_fraction_{bagging_fraction}'
print(model_desc)
lgbm = LGBM.LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=num_leaves,
                                learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)
lgbm.fit(train_x, train_y)
y_pred = lgbm.predict(val_x)
print(classification_report(y_true=val_y, y_pred=y_pred))
print(confusion_matrix(y_true=val_y, y_pred=y_pred))
print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


n_estimators_105-max_depth_6-num_leaves_50-learning_rate_0.2-bagging_fraction_0.85


LGBMClassifier(bagging_fraction=0.9, feature_fraction=0.9, learning_rate=0.2,
               max_depth=6, n_estimators=105, num_leaves=50, objective='binary',
               reg_lambda=0.2)

              precision    recall  f1-score   support

           0       0.90      0.97      0.93     21729
           1       0.70      0.44      0.54      4010

    accuracy                           0.88     25739
   macro avg       0.80      0.70      0.74     25739
weighted avg       0.87      0.88      0.87     25739

[[20969   760]
 [ 2234  1776]]
AUC score:  0.7039582345622437
MCC score:  0.4963826593427266


- SVM
    - https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769

In [8]:
produce_date = 200808
model = 'SVM'
kernels = ['linear', 'rbf', 'poly']  #linear
gammas = [0.1, 1, 10, 100]
Cs = [0.1, 1, 10, 100, 1000]
degrees = [0, 1, 2, 3, 4, 5, 6]
feature_desc = 'EDA_reference_featureOpt_standardlizedScaled'


In [ ]:
# for C in Cs:
kernel = kernels[0]
gamma = 'auto'
C = 1
degree = 3
model_desc = f'kernel_{kernel}-gamma_{gamma}-C_{C}-degree_{degree}'
print(model_desc)
svm = SVC(kernel=kernel,
          gamma=gamma,
          C=C,
          degree=degree,
          random_state=39)
svm.fit(train_x, train_y)
y_pred = svm.predict(val_x)
print(classification_report(y_true=val_y, y_pred=y_pred))
print(confusion_matrix(y_true=val_y, y_pred=y_pred))
print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'
    

kernel_linear-gamma_auto-C_1-degree_3


In [ ]:
# y_pred = svm.predict(val_x)
# print(classification_report(y_true=val_y, y_pred=y_pred))
# print(confusion_matrix(y_true=val_y, y_pred=y_pred))
# print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
# print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))
# model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=val_y, y_pred=y_pred), 4)}'


#### prediction

In [20]:
# test_x = test_target
test_x.shape

### Random Forest ###
# target_pred = RFC.predict(test_x)
### Gradient Boosting ###
target_pred = gbc.predict(test_x)
### XGBoost ###
# target_pred = XGB.predict(test_x)
### LGBM ###
# target_pred = lgbm.predict(test_x)
### SVM ###
# target_pred = svm.predict(test_x)

submitSample['open_flag'] = target_pred
submitSample.head()
submitSample.groupby(['open_flag']).size()


(55970, 33)

,row_id,open_flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


open_flag
0    51285
1     4685
dtype: int64

In [43]:
### some conditions quick tests

open_1_rowid = train_userAttr[(train_userAttr['open_count_trend_30-10']>0)|(train_userAttr['open_count_trend_60-30']>0)].row_id.values
open_1_rowid

sub_open_flag = []
for i in range(0, len(train_userAttr)):
    if i in open_1_rowid:
        sub_open_flag.append(1)
    else:
        sub_open_flag.append(0)

print(classification_report(y_true=train_userAttr.open_flag, y_pred=sub_open_flag))
print(confusion_matrix(y_true=train_userAttr.open_flag, y_pred=sub_open_flag))
print('AUC score: ', roc_auc_score(y_true=train_userAttr.open_flag, y_score=sub_open_flag))
print('MCC score: ', matthews_corrcoef(y_true=train_userAttr.open_flag, y_pred=sub_open_flag))

submitSample = pd.read_csv('sample_submission_0_1.csv')
submitSample['row_id'] = test['row_id']
submitSample.shape
submitSample.info()
submitSample['open_flag'] = 0
submitSample['open_flag'] = np.where(submitSample['row_id'].isin(open_1_rowid), 1, 0)
submitSample.head()
submitSample.groupby(['open_flag']).size()
submitSample.to_csv(f'submission__200803_open_count_trend_30-10_condition.csv', index=False)


array([    1,     5,     8, ..., 73534, 73537, 73538], dtype=int64)

              precision    recall  f1-score   support

           0       0.93      0.60      0.73     62083
           1       0.26      0.75      0.39     11456

    accuracy                           0.63     73539
   macro avg       0.59      0.68      0.56     73539
weighted avg       0.82      0.63      0.68     73539

[[37509 24574]
 [ 2857  8599]]
AUC score:  0.677393044746504
MCC score:  0.25856468082999745


(55970, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55970 entries, 0 to 55969
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   row_id     55970 non-null  int64
 1   open_flag  55970 non-null  int64
dtypes: int64(2)
memory usage: 874.7 KB


,row_id,open_flag
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


open_flag
0    30209
1    25761
dtype: int64

In [21]:
submitSample.to_csv(f'submission_{produce_date}__{model}_{model_desc}_feature-{feature_desc}.csv', index=False)
